In [13]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import matplotlib.pyplot as plt

import time
import json
import sys
sys.path.append("../../")

from src.context_aware.config import ModelConfig, DatasetConfig
from src.dataset_manager import DatasetConvertor 
from src.context_aware.preprocessing import PreparingDataset
from src.context_aware.models import createModel
from src.context_aware.training import createDataLoaders
from src.context_aware.evaluation import evaluateModel


In [2]:
rawDatasetFolder = "../../data/raw"
model_folder = "../../data/models/context_aware"
configPath = "../config/conbined_flows.json"

direction = "forward"
lenWindow = 20
verbose = True

In [3]:
config = json.load(open(configPath, "r"))
mapping = config.get("DIRECTION_MAPPING", {})
dpdr_params = config.get("dpdr_params", {})

datasetConverter = DatasetConvertor(rawDatasetFolder, config)
datasetConverter.processDataset(direction=direction, **dpdr_params[direction])
dataUnit = datasetConverter.getDataUnit(f"combined_{mapping[direction]}")

datasetConfig = DatasetConfig.initialize(len_window=lenWindow, data_augment=False)
_, testData = PreparingDataset(dataUnit, datasetConfig, verbose=False)

modelConfig = ModelConfig.from_dataset(datasetConfig, testData)
model, device = createModel(modelConfig)
model.load_checkpoint(f"{model_folder}/direction_{direction}_lenWindow_{lenWindow}.pth")

========== Combined ============
Forward: Compression rate: 0.2328


In [6]:
result = evaluateModel(model, testData)

In [16]:
import torch

tensor_ = [torch.stack([torch.from_numpy(d).float() for d in data]) for data in testData[0]]

'''
start_time = time.time()
output = model.forward_from_batchdata(batch)
end_time = time.time()
inference_time = end_time - start_time
print(f"Inference time: {inference_time:.6f} seconds")
'''

'\nstart_time = time.time()\noutput = model.forward_from_batchdata(batch)\nend_time = time.time()\ninference_time = end_time - start_time\nprint(f"Inference time: {inference_time:.6f} seconds")\n'